In [3]:
import os
import urllib.request
import json
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

from reportlab.pdfgen import canvas
from reportlab.platypus import (SimpleDocTemplate, Paragraph, PageBreak, Image, Spacer, Table, TableStyle, KeepInFrame, Frame)
from reportlab.lib.enums import TA_LEFT, TA_RIGHT, TA_CENTER, TA_JUSTIFY
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib.pagesizes import LETTER, inch
from reportlab.graphics.shapes import Line, LineShape, Drawing
from reportlab.lib.colors import Color
from reportlab.pdfbase.acroform import AcroForm
from reportlab.lib import colors
from reportlab.platypus.flowables import Flowable
import PIL

In [4]:
def import_json_to_dict(url) :
    response = urllib.request.urlopen(url)
    my_dict = json.loads(response.read())
    return my_dict

In [5]:
def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)

In [6]:
def format_amount(indic, valeur):
    if "Montant" in indic:
        f_valeur = float(valeur)
        if f_valeur > 1000000:
            return str(round(f_valeur/1000000, 1)) + ' M€'
        elif f_valeur > 10000:
            return str(round(f_valeur/1000, 1)) + ' k€'
        else:
            return str(f_valeur)
    else:
        try:
            return int(valeur.split(".")[0])
        except ValueError as err:
            print(f"L'indicateur {indic} possède des valeurs invalides : {err}")

In [7]:
mailles = ["national", "regional", "departemental"]

In [8]:
pp_dep = pd.read_csv("pp_dep.csv", sep=";", dtype={"reg":str, "dep":str})

In [9]:
pp_dep[pp_dep['departement'] == 'Etranger']

,Unnamed: 0,indicateur,state_id,period_date,period_month_tri,Date,valeur,departement,mesure,short_indic,short_mesure,dep,reg,libelle,region
94,94,Nombre d’entrées en service civique - SCI,Valeur Actuelle,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,473.0,Etranger,Service civique,Nombre d'entrées en service civique,Service civique,00,00,Etranger,Etranger
202,202,Nombre d’entrées en service civique - SCI,Valeur Actuelle,2020-11-30T00:00:00.0000000,Nov,Novembre 2020,385.0,Etranger,Service civique,Nombre d'entrées en service civique,Service civique,00,00,Etranger,Etranger
310,310,Nombre d’entrées en service civique - SCI,Valeur Actuelle,2020-12-31T00:00:00.0000000,Dec,Décembre 2020,421.0,Etranger,Service civique,Nombre d'entrées en service civique,Service civique,00,00,Etranger,Etranger
418,418,Nombre d’entrées en service civique - SCI,Valeur Actuelle,2021-02-28T00:00:00.0000000,Feb,Février 2021,492.0,Etranger,Service civique,Nombre d'entrées en service civique,Service civique,00,00,Etranger,Etranger
950,950,Nombre d’entreprises bénéficiaires du disposit...,Valeur Actuelle,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,0.0,Etranger,Soutien à la modernisation industrielle et ren...,Nombre d'entreprises bénéficiaires du dispositif,AAP industrie : modernisation industrielle et ...,00,00,Etranger,Etranger
1060,1060,Nombre de PME bénéficiaires - MIR,Valeur Actuelle,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,0.0,Etranger,Soutien à la modernisation industrielle et ren...,Nombre de PME bénéficiaires,AAP industrie : modernisation industrielle et ...,00,00,Etranger,Etranger
1471,1471,Nombre d'accompagnements dispensés - FUM,Valeur Actuelle,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,0.0,Etranger,France Num,Nombre d'accompagnements dispensés,"France Num : aide à la numérisation des TPE,PM...",00,00,Etranger,Etranger
1657,1657,Montant total des projets - SAC,Valeur Actuelle,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,0.0,Etranger,Relocalisation : sécurisation des approvisionn...,Montant total des projets,AAP Industrie : Sécurisation approvisionnement...,00,00,Etranger,Etranger
1767,1767,Nombre de TPE/PME/ETI bénéficiaires - SAC,Valeur Actuelle,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,0.0,Etranger,Relocalisation : sécurisation des approvisionn...,"Nombre de TPE,PME,ETI bénéficiaires",AAP Industrie : Sécurisation approvisionnement...,00,00,Etranger,Etranger
1877,1877,Nombre d’entreprises bénéficiaires - SAC,Valeur Actuelle,2021-01-31T00:00:00.0000000,Jan,Janvier 2021,0.0,Etranger,Relocalisation : sécurisation des approvisionn...,Nombre d'entreprises bénéficiaires,AAP Industrie : Sécurisation approvisionnement...,00,00,Etranger,Etranger


In [10]:
list_mesure_indic = list(pp_dep.pivot_table(index=["short_mesure", "short_indic"], values="valeur").index)

In [11]:
short_mesures_to_keep =set([
'AAP et AMI Efficacité énergétique',
'Assurance prospection',
"Ma Prime Rénov'",
"Prime à l'embauche des jeunes",
"Prime à l'embauche pour les travailleurs handicapés",
"Apprentissage",
"Bonus électrique",
"Contrats Initiatives Emploi (CIE) Jeunes",
'Contrats de professionnalisation',
'France Num : aide à la numérisation des TPE,PME,ETI',
'Garantie jeunes',
"AAP industrie : Modernisation des filières auto et aéro",
"Parcours emploi compétences (PEC) Jeunes",
'Prime à la conversion des véhicules légers',
"AAP Industrie : Soutien aux projets industriels territoires",
"AAP Industrie : Sécurisation approvisionnements critiques",
"Renforcement subventions Business France",
'Rénovation des bâtiments Etats (marchés notifiés)',
"Service civique",
'Soutien recherche aéronautique civil'
])
#print(list_mesure_indic)
#print(list(x[0] for x in list_mesure_indic if not(x[0] in short_mesures_to_keep)))
list_mesure_indic = [x for x in list_mesure_indic if x[0] in short_mesures_to_keep]

In [12]:
pp_dep[pp_dep['reg'].isnull()]

,Unnamed: 0,indicateur,state_id,period_date,period_month_tri,Date,valeur,departement,mesure,short_indic,short_mesure,dep,reg,libelle,region


In [13]:
pd.pivot_table(pp_dep, index=["mesure","short_mesure", "reg","region", "Date", "period_date", "short_indic"], values="valeur", aggfunc=np.sum).reset_index()

,mesure,short_mesure,reg,region,Date,period_date,short_indic,valeur
0,Appels à projets pour les projets d'efficacité...,AAP et AMI Efficacité énergétique,01,Guadeloupe,Décembre 2020,2020-12-31T00:00:00.0000000,Emissions de gaz à effet de serre évitées,0.0
1,Appels à projets pour les projets d'efficacité...,AAP et AMI Efficacité énergétique,01,Guadeloupe,Décembre 2020,2020-12-31T00:00:00.0000000,Montant cumulé de l'investissement total ainsi...,0.0
2,Appels à projets pour les projets d'efficacité...,AAP et AMI Efficacité énergétique,01,Guadeloupe,Décembre 2020,2020-12-31T00:00:00.0000000,Nombre d'entreprises ayant reçu l'aide,0.0
3,Appels à projets pour les projets d'efficacité...,AAP et AMI Efficacité énergétique,01,Guadeloupe,Février 2021,2021-02-28T00:00:00.0000000,Emissions de gaz à effet de serre évitées,0.0
4,Appels à projets pour les projets d'efficacité...,AAP et AMI Efficacité énergétique,01,Guadeloupe,Février 2021,2021-02-28T00:00:00.0000000,Montant cumulé de l'investissement total ainsi...,0.0
...,...,...,...,...,...,...,...,...
3301,"Écosystèmes territoriaux hydrogène DGEC, AAP A...","Écosystèmes territoriaux hydrogène DGEC, AAP A...",988,Nouvelle-Calédonie,Février 2021,2021-02-28T00:00:00.0000000,Nombre de projets soutenus,0.0
3302,"Écosystèmes territoriaux hydrogène DGEC, AAP A...","Écosystèmes territoriaux hydrogène DGEC, AAP A...",989,Île de Clipperton,Février 2021,2021-02-28T00:00:00.0000000,Montant total de CO2 évité,0.0
3303,"Écosystèmes territoriaux hydrogène DGEC, AAP A...","Écosystèmes territoriaux hydrogène DGEC, AAP A...",989,Île de Clipperton,Février 2021,2021-02-28T00:00:00.0000000,Nombre de projets soutenus,0.0
3304,"Écosystèmes territoriaux hydrogène DGEC, AAP A...","Écosystèmes territoriaux hydrogène DGEC, AAP A...",COM,Collectivités d’outre-mer,Février 2021,2021-02-28T00:00:00.0000000,Montant total de CO2 évité,0.0


In [14]:
pp_reg = pd.pivot_table(pp_dep, index=["mesure","short_mesure", "reg","region", "Date", "period_date", "short_indic"], values="valeur", aggfunc=np.sum)
pp_reg.rename(columns={"reg":"libelle"}, inplace=True)
pp_reg.reset_index(inplace=True)

In [15]:
pp_dep['region'].unique().shape

(23,)

In [16]:
'Etranger' in pp_reg['region'].unique()

True

In [17]:
pp_nat = pd.pivot_table(pp_reg, index=["mesure", "short_mesure", "Date","period_date", "short_indic"], values="valeur", aggfunc=np.sum)
pp_nat.reset_index(inplace=True)

In [18]:
pp_nat[pp_nat['mesure'] == 'Service civique']

,mesure,short_mesure,Date,period_date,short_indic,valeur
139,Service civique,Service civique,Décembre 2020,2020-12-31T00:00:00.0000000,Nombre d'entrées en service civique,54519.0
140,Service civique,Service civique,Février 2021,2021-02-28T00:00:00.0000000,Nombre d'entrées en service civique,66881.0
141,Service civique,Service civique,Janvier 2021,2021-01-31T00:00:00.0000000,Nombre d'entrées en service civique,63360.0
142,Service civique,Service civique,Novembre 2020,2020-11-30T00:00:00.0000000,Nombre d'entrées en service civique,42694.0


In [19]:
dict_mesure_indic = {}
for x in list_mesure_indic:
    if x[0] in dict_mesure_indic:
        dict_mesure_indic[x[0]].append(x[1])
    else:
        dict_mesure_indic[x[0]] = [x[1]]

In [20]:
dict_mesure_indic['AAP Industrie : Soutien aux projets industriels territoires'] = ['Nombre de TPE,PME,ETI bénéficiaires']
dict_mesure_indic['AAP Industrie : Sécurisation approvisionnements critiques'] = ['Nombre de TPE,PME,ETI bénéficiaires']
dict_mesure_indic['AAP et AMI Efficacité énergétique'] = ["Nombre d'entreprises ayant reçu l'aide"]
dict_mesure_indic['AAP industrie : Modernisation des filières auto et aéro'] = ['Nombre de PME']

In [21]:
# On ne veut pas afficher les lignes de Prime Rénov nulles
pp_dep = pp_dep.loc[(pp_dep.short_mesure != "Ma Prime Rénov'") | (pp_dep.valeur != 0) ]

In [22]:
pp_dep.valeur = pp_dep.valeur.astype(str)
pp_dep.valeur = pp_dep.apply(lambda x: format_amount(x["short_indic"], x["valeur"]), axis=1)

In [23]:
pp_reg.valeur = pp_reg.valeur.astype(str)
pp_reg.valeur = pp_reg.apply(lambda x: format_amount(x["short_indic"], x["valeur"]), axis=1)

In [24]:
pp_nat.valeur = pp_nat.valeur.astype(str)
pp_nat.valeur = pp_nat.apply(lambda x: format_amount(x["short_indic"], x["valeur"]), axis=1)

In [25]:
taxo_dep_df = pd.read_csv('refs/taxo_deps.csv', dtype={'dep':str, 'reg':str})
taxo_dep_df['dep'] = taxo_dep_df['dep'].apply(lambda x: x.zfill(2))
taxo_dep_df['reg'] = taxo_dep_df['reg'].apply(lambda x: x.zfill(2))
dep_list = list(taxo_dep_df['dep'].unique())
print('{} departements.'.format(len(dep_list)))

taxo_reg_df = pd.read_csv('refs/taxo_regions.csv', dtype={'reg':str})
taxo_reg_df['reg'] = taxo_reg_df['reg'].apply(lambda x: x.zfill(2))
reg_list = list(taxo_reg_df['reg'].unique())
print('{} regions.'.format(len(reg_list)))

110 departements.
23 regions.


In [26]:
#Building the folders structures
img_dir_path = './img/'
reports_dir_path = './reports/'
mkdir_ifnotexist(reports_dir_path)

In [27]:
all_charts_as_df = {"departemental": {dep: {} for dep in dep_list},
                    "national": {'France': {}},
                    "regional": {reg: {} for reg in reg_list}}
    

In [28]:
def make_pp_chart(maille, mesure, short_indics):
    if maille == "departemental":
        df = pp_dep.loc[(pp_dep.short_mesure == mesure)].sort_values(by="period_date", ascending=True).copy()
        deps = taxo_dep_df.dep.unique()  # Liste exhaustive de départements
        
        default = df.groupby(["Date", "period_date"]).sum().sort_values("period_date", ascending=True).reset_index()
        default[short_indics] = '-'
        default = default[["Date"] + short_indics]

        for dep in deps:
            print(f"Plotting {mesure}-{short_indics} : departement {dep}")
            df_dep = df.loc[df.dep == dep]
            if df_dep.shape[0] == 0:
                all_charts_as_df[maille][dep][mesure] = default.T.reset_index().T
            else:
                df_plot = pd.pivot_table(df_dep, index=['period_date', 'Date'], columns=['short_indic'], values='valeur', aggfunc='first')
                df_plot = df_plot.reset_index().drop('period_date', axis=1)
                df_plot = df_plot.rename_axis(None, axis=1)
                df_plot = df_plot.fillna('-')
                df_plot = df_plot[['Date'] + short_indics]
                all_charts_as_df[maille][dep][mesure] = df_plot.T.reset_index().T
                
            
    elif maille == "regional":
        df = pp_reg.loc[(pp_reg.short_mesure == mesure)].sort_values(by="period_date", ascending=True).copy()
        regs = taxo_dep_df.reg.unique()
        
        default = df.groupby(["Date", "period_date"]).sum().sort_values("period_date", ascending=True).reset_index()
        default[short_indics] = '-'
        default = default[["Date"] + short_indics]

        for reg in regs:
            print(f"Plotting region {mesure}-{short_indics} : {reg}")
            df_reg = df.loc[df.reg == reg]
            if df_reg.shape[0] == 0:
                all_charts_as_df[maille][reg][mesure] = default.T.reset_index().T
            else:
                df_plot = pd.pivot_table(df_reg, index=['period_date', 'Date'], columns=['short_indic'], values='valeur', aggfunc='first')
                df_plot = df_plot.reset_index().drop('period_date', axis=1)
                df_plot = df_plot.rename_axis(None, axis=1)
                df_plot = df_plot.fillna('-')
                df_plot = df_plot[['Date'] + short_indics]
                all_charts_as_df[maille][reg][mesure] = df_plot.T.reset_index().T
            
    elif maille == "national":
        print(f"Plotting country {mesure}-{short_indics}")
        df_nat = pp_nat.loc[(pp_nat.short_mesure == mesure)].sort_values(by="period_date", ascending=True).copy()
        df_plot = pd.pivot_table(df_nat, index=['period_date', 'Date'], columns=['short_indic'], values='valeur', aggfunc='first')
        df_plot = df_plot.reset_index().drop('period_date', axis=1)
        df_plot = df_plot.rename_axis(None, axis=1)
        df_plot = df_plot.fillna('-')
        df_plot = df_plot[['Date'] + short_indics]
        all_charts_as_df[maille]['France'][mesure] = df_plot.T.reset_index().T


In [29]:
def format_col_labels(col_names_src, max_num_lines=3, num_char_threshold=14):
    """ Ajoute des retours à la ligne aux noms de colonnes pour réduire la largeur du texte 
    dans l'image. 
    :param str col_names_src: Liste des noms de colonnes.
    :param int max_num_lines: Nombre de lignes max. On n'ajoute plus de retour à la ligne par la suite.
    :param int num_char_threshold: Un retour à la ligne est ajouté dès lors que ce nombre de chars est atteint.
    """
    col_names = []
    for col_name_src in col_names_src:
        col_name = ''
        num_lines = 1
        num_chars_per_line = 0
        for word in col_name_src.split(' '):
            col_name += (word + ' ')
            num_chars_per_line += (len(word) + 1)
            if num_chars_per_line >= num_char_threshold and num_lines < max_num_lines:
                col_name += '\n'
                num_lines += 1
                num_chars_per_line = 0
        col_names.append(col_name.strip())
    return col_names


In [30]:
def make_all_charts():
    for short_mesure in dict_mesure_indic:
        short_indics = dict_mesure_indic[short_mesure]
        for maille in mailles :
            make_pp_chart(maille, short_mesure, short_indics)

In [31]:
make_all_charts()

Plotting country AAP Industrie : Soutien aux projets industriels territoires-['Nombre de TPE,PME,ETI bénéficiaires']
Plotting region AAP Industrie : Soutien aux projets industriels territoires-['Nombre de TPE,PME,ETI bénéficiaires'] : 84
Plotting region AAP Industrie : Soutien aux projets industriels territoires-['Nombre de TPE,PME,ETI bénéficiaires'] : 32
Plotting region AAP Industrie : Soutien aux projets industriels territoires-['Nombre de TPE,PME,ETI bénéficiaires'] : 93
Plotting region AAP Industrie : Soutien aux projets industriels territoires-['Nombre de TPE,PME,ETI bénéficiaires'] : 44
Plotting region AAP Industrie : Soutien aux projets industriels territoires-['Nombre de TPE,PME,ETI bénéficiaires'] : 76
Plotting region AAP Industrie : Soutien aux projets industriels territoires-['Nombre de TPE,PME,ETI bénéficiaires'] : 28
Plotting region AAP Industrie : Soutien aux projets industriels territoires-['Nombre de TPE,PME,ETI bénéficiaires'] : 75
Plotting region AAP Industrie : Sout

In [32]:
volet2mesures = {
'Ecologie': ["Ma Prime Rénov'",
		"Bonus électrique",
		'AAP et AMI Efficacité énergétique',
		'Prime à la conversion des véhicules légers',
		'Soutien recherche aéronautique civil', 
		'Rénovation des bâtiments Etats (marchés notifiés)',],

'Compétitivité': ['Assurance prospection', 
		'France Num : aide à la numérisation des TPE,PME,ETI',
		"AAP Industrie : Soutien aux projets industriels territoires",
		"AAP Industrie : Sécurisation approvisionnements critiques",
		"AAP industrie : Modernisation des filières auto et aéro",
		"Renforcement subventions Business France",],

'Cohésion': ["Apprentissage",
		"Prime à l'embauche des jeunes",
		"Prime à l'embauche pour les travailleurs handicapés",
		"Contrats Initiatives Emploi (CIE) Jeunes",
		'Contrats de professionnalisation',
		'Garantie jeunes',
		"Parcours emploi compétences (PEC) Jeunes",
		"Service civique",]

}

In [33]:
def get_kpi(dep, short_indic, short_mesure):
    kpi_dep = (pp_dep.loc[(pp_dep.dep == dep) 
                          & (pp_dep.short_mesure == short_mesure) 
                          & (pp_dep.short_indic == short_indic)]
                .sort_values(by="period_date", ascending=False))
    if kpi_dep.shape[0] != 0:
        date= kpi_dep.iloc[0].Date
        valeur = kpi_dep.iloc[0].valeur
    else:
        date = pp_dep.Date.max()
        valeur = 0
    return date, valeur



class ReportCanvas(canvas.Canvas):

    def __init__(self, *args, **kwargs):
        canvas.Canvas.__init__(self, *args, **kwargs)
        self.pages = []
        self.width, self.height = LETTER
        self.flag = False

    def showPage(self):
        self.pages.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        page_count = len(self.pages)
        for page in self.pages:
            self.__dict__.update(page)
            self.draw_canvas(page_count, self._pageNumber)
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)

    def draw_canvas(self, page_count, page_number):
        page = f"Page {self._pageNumber} sur {page_count}"
        x = 128
        self.saveState()
        self.setStrokeColorRGB(0, 0, 0)
        self.setLineWidth(0.5)
        self.drawImage("img/LogoRF.png", self.width - inch * 8 - 5, self.height - 120, width=100, height=100,
                       preserveAspectRatio=True)
        self.drawImage("img/logo_Francerelance_1321294.54.png", self.width - inch * 2, self.height - 130, width=130, height=130,
                       preserveAspectRatio=True)
        if page_number > 1:
            self.line(130, 680, LETTER[0] - 130, 680)  # Séparateur haut-de-page
            self.line(66, 78, LETTER[0] - 66, 78)  # Séparateur pied-de-page
            self.setFont('Times-Roman', 10)
            self.drawString(LETTER[0] - x, 65, page)
        self.restoreState()
        
        
class InteractiveTextfield(Flowable):
    def __init__(self, x, y, width, height, name='textfield'):
        Flowable.__init__(self)
        self.name = name
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def draw(self):
        self.canv.saveState()
        form = self.canv.acroForm
        form.textfield(name=self.name, relative=True, 
                       x=self.x, y=self.y, width=self.width, height=self.height,
                      maxlen=255, value='Entrez un commentaire...', fillColor=colors.white, )
        self.canv.restoreState()
        return


class PDFReport:
    
    def __init__(self, path, code_dep):
        self.path = path
        self.styleSheet = getSampleStyleSheet()
        self.elements = []  # Story
        self.dep = code_dep
        self.dep_name = taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['libelle']
        self.font = 'Arial'
        self.page_count = 0  # Increment 

    
    def create_front_page(self):
        # All texts
        text_title = "<b>SUIVI TERRITORIAL <br/> DU PLAN DE RELANCE</b>"
        text_subtitle = f"<i>Données pour le département : {self.dep_name}</i>"
        text_date = f"<i>Date : {datetime.datetime.today().strftime('%Y-%m-%d')}</i>"
        
        self.elements.append(Spacer(30, 150))  # Décale le titre vers le bas
        
        # Titre du rapport
        title_style = ParagraphStyle(self.font, fontSize=30, leading=60, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        title = Paragraph(text_title, title_style)
        self.elements.append(title)
        
        self.elements.append(Spacer(0, 30))
        
        # Sous-titre avec le nom du département
        subtitle_style = ParagraphStyle(self.font, fontSize=20, leading=50, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        subtitle = Paragraph(text_subtitle, subtitle_style)
        self.elements.append(subtitle)
        
        # Date de création du rapport
        date_style = ParagraphStyle(self.font, fontSize=20, leading=50, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        date = Paragraph(text_date, date_style)
        self.elements.append(date)
        
        self.elements.append(PageBreak())
        self.page_count += 1
        
        
    def create_volet_page(self, volet):
        # All texts
        text_title = f"<b>Volet : {volet}</b>"

        self.elements.append(Spacer(30, 150))  # Décale le titre vers le bas
        
        # Titre du rapport
        title_style = ParagraphStyle(self.font, fontSize=30, leading=60, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        title = Paragraph(text_title, title_style)
        self.elements.append(title)
        
        self.elements.append(PageBreak())
        self.page_count += 1
    
    
    def create_content_page(self, short_mesure, volet):
        reg = taxo_dep_df[taxo_dep_df['dep'] == self.dep].iloc[0]['reg']
        reg_name = taxo_reg_df[taxo_reg_df['reg'] == reg].iloc[0]['libelle']
        
        # Affichage mesure
        text_mesure = f"<b>{short_mesure}</b>"
        mesure_style = ParagraphStyle(self.font, fontSize=15, leading=15, justifyBreaks=1, alignment=TA_CENTER, justifyLastLine=1)
        mesure = Paragraph(text_mesure, mesure_style)
        block_mesure = KeepInFrame(288, 576, [mesure], hAlign='CENTER')  # Pour créer des retours à la ligne en cas de textes longs
        self.elements.append(block_mesure)
        self.elements.append(Spacer(1, 60))
        
        # Affichage phrase indic
        short_indic = dict_mesure_indic[short_mesure][0]
        date, valeur = get_kpi(self.dep, short_indic, short_mesure)
        text_indic = f"En {date}, {short_indic} : {valeur} (données cumulées)"
        indic_style = ParagraphStyle(self.font, fontSize=11, leading=20, justifyBreaks=1, alignment=TA_LEFT, justifyLastLine=1)
        sentence_indic = Paragraph(text_indic, indic_style)
        self.elements.append(sentence_indic)
        
        # Affichage par niveau de localité --> Table
        text_national = f"Niveau National"
        df_mesure_national = all_charts_as_df['national']['France'][short_mesure]
        title_national, table_national = self._get_block_indic_for_locality(text_national, df_mesure_national)
        
        text_regional = f"Niveau Régional : {reg_name}"
        df_mesure_regional = all_charts_as_df['regional'][reg][short_mesure]
        title_regional, table_regional = self._get_block_indic_for_locality(text_regional, df_mesure_regional)
        
        text_departemental = f"Niveau Départemental : {self.dep_name}"
        df_mesure_departemental = all_charts_as_df['departemental'][dep][short_mesure]
        title_departemental, table_departemental = self._get_block_indic_for_locality(text_departemental, df_mesure_departemental)
    
        data_table = [  [text_national,],
                        [table_national, ],
                        [text_regional, ],
                        [table_regional, ],
                        [text_departemental, ],
                        [table_departemental,],]

        indic_cell_height = 130
        title_cell_height = 20
        row_table_heights = [title_cell_height, indic_cell_height, 
                             title_cell_height, indic_cell_height, 
                             title_cell_height, indic_cell_height, 
                            ]
        style = TableStyle(
            [
                ('FONTNAME', (0,0), (0,-1), 'Helvetica-Bold'),
                ('ALIGN', (0,0), (0,-1), 'CENTER'),
                ('VALIGN',(0,0),(0,-1),'TOP'),
            ]
        )
        table = Table(data_table, colWidths=10, rowHeights=row_table_heights, hAlign='CENTER', style=style)
        self.elements.append(table)
        
        form_textfield = InteractiveTextfield(x=0, y=-10, width=LETTER[0]-132, height=50, name=f"textfield{self.page_count}")
        self.elements.append(form_textfield)

        self.elements.append(PageBreak())
        self.page_count += 1
        
    def _get_block_indic_for_locality(self, text_title, df_indics):
        # Pour l'affichage titre
        title = Paragraph(text_title)

        # Pour l'affichage de la table
        blue_header = colors.Color(red=0,green=0,blue=0.3, alpha=0.75)
        grey_color = colors.Color(red=0, green=0, blue=0, alpha=0.1)
        
        # Style pour paragraphe
        style_header = ParagraphStyle('header', fontSize=10, wordWrap='CJK', alignment=TA_CENTER, fontName='Helvetica-Bold')
        style_content = ParagraphStyle('content', fontSize=10, alignment=TA_CENTER)
        text_table_content = []
        for i, row in enumerate(df_indics.values):
            text_table_content.append([])
            for cell in row:
                if i == 0:
                    text_table_content[-1].append(Paragraph(f'<font color={colors.white}>{cell}</font>', style_header))
                else:
                    text_table_content[-1].append(Paragraph(str(cell), style_content))
                
        # Style pour tableau
        table_style = [
            ('BACKGROUND', (0,0), (-1,0), blue_header),  # Header
            ('VALIGN', (0,0), (-1,-1), 'TOP')
        ]
        for i in range(1, df_indics.shape[0]):
            if i % 2 == 0:
                table_style.append(('BACKGROUND', (0,i), (-1,i), grey_color))
            else:
                table_style.append(('BACKGROUND', (0,i), (-1,i), colors.white),)
        table_style = TableStyle(table_style)
        table = Table(text_table_content, colWidths=120, style=table_style)
        return title, table

    def save_report(self):
        self.doc = SimpleDocTemplate(self.path, pagesize=LETTER, topMargin=50)
        self.doc.multiBuild(self.elements, canvasmaker=ReportCanvas)


def create_single_report(dep, report_path):
    """ Créer un document PDF concernant le departement indiqué.
    :param dep code département (ex: 01, 72...)
    :param report_path
    """
    report = PDFReport(report_path, dep)
    
    report.create_front_page()

    for volet, short_mesures in volet2mesures.items():
        report.create_volet_page(volet)
        for short_mesure in sorted(short_mesures):
            report.create_content_page(short_mesure, volet)

    report.save_report()


# TEST
#dep = '2A'
#report_path = 'test_report.pdf'
#create_single_report(dep, report_path)

In [34]:
all_charts_as_df['departemental'].__len__()

110

In [35]:
all_charts_as_df['regional'].__len__()

23

In [39]:
mkdir_ifnotexist(os.path.join(reports_dir_path))

for i, dep in enumerate(taxo_dep_df.dep.unique(), start=1):
    departement = taxo_dep_df[taxo_dep_df['dep'] == dep].iloc[0]['libelle']
    report_path = os.path.join(reports_dir_path, 'Suivi_territorial_plan_relance_'+ departement+'.pdf')
    create_single_report(dep, report_path)
    
    print(str(i) + ' ' + str(datetime.datetime.today()) + ' - ' + dep + ' done.')

1 2021-03-30 09:00:39.185482 - 01 done.
2 2021-03-30 09:00:39.465605 - 02 done.
3 2021-03-30 09:00:39.752304 - 03 done.
4 2021-03-30 09:00:40.039397 - 04 done.
5 2021-03-30 09:00:40.325200 - 05 done.
6 2021-03-30 09:00:40.603590 - 06 done.
7 2021-03-30 09:00:40.927747 - 07 done.
8 2021-03-30 09:00:41.210959 - 08 done.
9 2021-03-30 09:00:41.495832 - 09 done.
10 2021-03-30 09:00:41.783753 - 10 done.
11 2021-03-30 09:00:42.064617 - 11 done.
12 2021-03-30 09:00:42.346871 - 12 done.
13 2021-03-30 09:00:42.655686 - 13 done.
14 2021-03-30 09:00:42.940323 - 14 done.
15 2021-03-30 09:00:43.224624 - 15 done.
16 2021-03-30 09:00:43.500760 - 16 done.
17 2021-03-30 09:00:43.781645 - 17 done.
18 2021-03-30 09:00:44.060335 - 18 done.
19 2021-03-30 09:00:44.337768 - 19 done.
20 2021-03-30 09:00:44.643753 - 21 done.
21 2021-03-30 09:00:44.923543 - 22 done.
22 2021-03-30 09:00:45.199998 - 23 done.
23 2021-03-30 09:00:45.475803 - 24 done.
24 2021-03-30 09:00:45.752285 - 25 done.
25 2021-03-30 09:00:46.06